In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
import glob

from sklearn.utils import shuffle
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend

In [36]:
import zipfile

In [38]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/nlp project/ezyzip.zip", 'r')
zip_ref.extractall("/temp")
zip_ref.close()

In [41]:
filelist = glob.glob("/temp/dat/*.txt")

In [48]:
filelist

[]

In [42]:
text_data = []
for file in filelist:
    with open(file, "r") as file:
      text_data.append(file.read())

In [43]:
text_data

[]

In [44]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)

In [45]:
word_idx = tokenizer.word_index #Last is the key
idx_word = tokenizer.index_word

In [46]:
word_counts = tokenizer.word_counts
num_words = len(word_counts)

In [2]:
pip install gensim

  Found existing installation: Cython 0.29.13
    Uninstalling Cython-0.29.13:
      Successfully uninstalled Cython-0.29.13
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import re
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer

D:\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [136]:
df1 = pd.read_csv('/content/drive/MyDrive/nlp project/TrainData.txt', sep='\t')

In [2]:
df1 = pd.read_csv('D:/Downloads/Python_GL/NLP/Hackathon/TrainData.txt', sep='\t')

In [3]:
df1.head()

,Answer,Question
0,Yes. It's absolutely beautiful today.,The weather is great isn't it?
1,"yes, i like that one, too.",that one. the one that's all black.
2,it's really nice.,i got it from macy's.
3,at 8:00 p.m.,when does it start?
4,"nothing, except my favorite color is blue.",what's the matter with green eyes?


In [138]:
df1.shape

(5495, 2)

In [139]:
data.shape

(4875, 2)

In [4]:
data=df1

In [5]:
data =data.drop_duplicates(subset='Question')
data.head(10)

,Answer,Question
0,Yes. It's absolutely beautiful today.,The weather is great isn't it?
1,"yes, i like that one, too.",that one. the one that's all black.
2,it's really nice.,i got it from macy's.
3,at 8:00 p.m.,when does it start?
4,"nothing, except my favorite color is blue.",what's the matter with green eyes?
5,I have read just about everything in Project ...,have you ever read a book
6,a fancy name for applied computer science in ...,what is bioinformatics
7,"Sure, the changing rooms are over there.",Can I try it on?
8,I want it taken from my checking account.,Which account are you making this withdrawal f...
9,and you get to play with a lot of dogs.,that's the truth.


In [6]:
data=data.dropna()

In [7]:
data.shape

(4875, 2)

In [8]:
stopwords_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def my_tokenizer(doc):
    words = word_tokenize(doc)
    pos_tags = pos_tag(words)
    non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    
    lemmas = []
    for w in non_punctuation:
        if w[1].startswith('J'):
            pos = wordnet.ADJ
        elif w[1].startswith('V'):
            pos = wordnet.VERB
        elif w[1].startswith('N'):
            pos = wordnet.NOUN
        elif w[1].startswith('R'):
            pos = wordnet.ADV
        else:
            pos = wordnet.NOUN
        
        lemmas.append(lemmatizer.lemmatize(w[0], pos))

    return lemmas

In [149]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [154]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [151]:
import nltk

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [9]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)
#tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(tuple(df1['Question']))
print(tfidf_matrix.shape)

(5495, 2333)


def question_answer(question):
    
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)
    similarity_max = np.argmax(similarity)
    
    print('\n> Similarity:\n- {:.2%}\n'.format(similarity[0, similarity_max]))
    print('> Nearest Question:\n- {}\n'.format(df1.iloc[similarity_max]['Question']))
    print('> Answer:\n- {}'.format(df1.iloc[similarity_max]['Answer']))

In [15]:
def question_answer(question): # modified the print statements to get only required output
    
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)
    similarity_max = np.argmax(similarity)
    
    print(df1.iloc[similarity_max]['Answer'])

In [11]:
import pandas as pd
test = pd.read_csv('TestData.csv')
test.head(10)

,I.D.,Question
0,QN_1,"i'll give you a speech like that, too."
1,QN_2,"i know, you're absolutely right."
2,QN_3,i liked it.
3,QN_4,the baby was eight pounds six ounces.
4,QN_5,I was sold a wireless service unavailable in m...
5,QN_6,maybe four or five pounds?
6,QN_7,do you know hal
7,QN_8,"Yeah,actually a lot of them are."
8,QN_9,"if it's old age, why don't both of your hands ..."
9,QN_10,it's supposed to start at about eight.


In [16]:
question_user = input('Type your question here: ')
question_answer(question_user)

Type your question here: i'll give you a speech like that, too
do you think anyone will come to my funeral?


In [31]:
b=test['Question'].values.tolist()

In [65]:
for x in b:
    if x!= None:
        answr_list= question_answer(x)
print(answr_list)

do you think anyone will come to my funeral?
i wish it would cool off one day.
Oh, yes, I really like it.
that's good to hear.
I see it here,we charged you $5 extra a month.
my waist is bigger than it was.
 hal is the famous artificial intelligence from "2001".
How does she act? 
that's a good question. maybe it's not old age.
how many invitations has she given out?
nothing really.
Yes, I am very excited. I could not forget the picnic I had yesterday.
The weather is hot.
they were on sale for $80.
that tasted so good.
It was really exciting. It was more enjoyable to me because I had never been to Garo Pahar.
 Europe
the next four years will be good years.
i'm doing well. how about you?
I agree.
 A secret organization believed by some to be in control of all governments through a worldwide conspiracy.
that's hard to take.
i love boiled peanuts.
use a tissue next time.
then stop eating the bread!
i think there are more fish.
i wasn't able to attend school because i was feeling ill.
i enj

Thankyou.
like real people with real problems.
 cancer.
well, it's your life.
no, my sister and i will travel together.
really? how many do you have?
No, it's too close to the salad bar.
and he poured water into our glasses.
I don't think here it is challenging enough for me.
 i'm probably not as sincere as i should be.
Yes, I can speak english
who cares? everyone is strange.
 work for free.  we don't need money.
i didn't do anything.
that sounds like a great job.
i was busy doing something. i apologize.
the league suspended him for 50 games.
she did? why didn't anyone tell me?
 Of what?
i am going to see a movie with a friend of mine. what about you?
well, i guess that's it.
i'm sorry to hear that. would you like the assignments from english class?
me too. it always smells so fresh after it rains.
i thought you just bought a pair.
what's that?
if we are bad, we will be unhappy forever in hell.
Not possible human
you need to get a lot of lessons when you're really young.
let me think a

i can add two and two.
so do i.
poodles bark a lot.
 i support the 2nd amendment.
no. i already have a pen.
your chances are very small.
lately, i've been quite busy.
You should do some exploring.
 I'm software.  I can't feel pain.
no, they smell like a thrift shop.
so?
You're right!Yale made a mistake rejecting me.
i don't see anything.
My developer only knows that
well, i hope it has a happy ending.
i don't have enough will power.
 That is a good story by Philip K. Dick.  Have you read VALIS or The Man In the High Castle?
from now on, let's bring our own sheets.
i think i'm going to explode.
i would rather deal with the winter than the summer.
did you take pictures at the world war ii monument?
i need pencils.
 A secret society that has supposedly existed for centuries.
are there going to be layoffs at this place?
I'm not.
where's the checkbook? i'm ready to rent it without even seeing it.
I'm majoring in philosophy.
 A secret society that has supposedly existed for centuries.
what d

In [67]:
list(answr_list)

TypeError: 'NoneType' object is not iterable

In [39]:
arr = []
for i in range(awr):
    arr.append(i)
print(arr)

TypeError: 'NoneType' object cannot be interpreted as an integer

In [158]:
question_user = input('Type your question here: ')
question_answer(question_user)

Type your question here: i'll give you a speech like that, too

> Similarity:
- 100.00%

> Nearest Question:
- i'll give you a speech like that, too.

> Answer:
- do you think anyone will come to my funeral?


In [158]:
question_user = input('Type your question here: ')
question_answer(question_user)

In [123]:
def clean_sentence(sentence,stopwords=False):
    sentence=sentence.lower().strip()
    sentence=re.sub(r'[^a-z0-9\s]',' ',sentence)
    if stopwords:
        sentence=remove_stopwords(sentence)
    return sentence

In [126]:
def get_cleaned_sentence(data,stopwords=False):
    sents=data[['Question']];
    cleaned_sentence=[]
    for index,row in data.iterrows():
        cleaned=clean_sentence(row['Question'],stopwords)
        cleaned_sentence.append(cleaned);
    return cleaned_sentence
cleaned_sentence=get_cleaned_sentence(data,stopwords=True)
print(cleaned_sentence);
print('\n')

['weather great isn t', 's black', 'got macy s', 'start', 's matter green eyes', 'read book', 'bioinformatics', 'try', 'account making withdrawal', 's truth', 'basketball', 'watch dvd', 'saw dad wipe nose sleeve yesterday', 'cancel right', 'm sure okay day', 've actually busy lately', 'wish free night m kind mad t', 'today', 'm going movies friend', '', 'drink', 'price stamps goes', 'forecast says warm weekend', 'long bus ride', 'movie', 'mixed', 'oh maybe called 911', 'll warmer walk', 'buy swiss cheese', 'maybe m sure', 'lost new pen', 'idea happy', 'things s polite nosey', 'survive old days', 'time', 'watch rerun', 'house expensive', 'lock doors', 'thinks stress', 'd s dirty m pretty hungry right', 's comfortable', 'tell dreams', 'm divorced', 'money like withdraw', 't idea', 'rich people know', 'tell gossips anybody', 'invented computers', 'mean', '', 'purpose meeting exactly', 'stock market', 'll buy', 'uses super computers', 'like', 'chances small', '50 million people voted mccai

In [127]:
clean_sentence_with_stopwords=get_cleaned_sentence(data,stopwords=False)
print(clean_sentence_with_stopwords)

['the weather is great isn t it ', 'that one  the one that s all black ', 'i got it from macy s ', 'when does it start ', 'what s the matter with green eyes ', 'have you ever read a book', 'what is bioinformatics', 'can i try it on ', 'which account are you making this withdrawal from ', 'that s the truth ', 'what is basketball', 'we can watch my dvd ', 'i saw dad wipe his nose on his sleeve yesterday ', 'i can cancel it for you right now ', 'i m sure everything will be okay in a day or two ', 'i ve actually been busy lately ', 'i wish i was free that night  i m kind of mad that i didn t go ', 'how are you doing today ', 'i m going to the movies with a friend  how about you ', 'what are they ', 'do you drink', 'the price of stamps goes up and up ', 'the forecast says that it will be warm on the weekend ', 'how long is this bus ride ', 'how was the movie ', 'i mixed them together ', 'oh  really  maybe you should have called 911 ', 'we ll get warmer as we walk ', 'i never used to buy swi

In [128]:
sentences=clean_sentence_with_stopwords
#split the word with white spaces
sentence_words=[[word for word in  document.split()]for document in sentences]
#convert the word to the dictonery
dictionary=corpora.Dictionary(sentence_words)
for key,value in dictionary.items():
    print(key,':',value)

0 : great
1 : is
2 : isn
3 : it
4 : t
5 : the
6 : weather
7 : all
8 : black
9 : one
10 : s
11 : that
12 : from
13 : got
14 : i
15 : macy
16 : does
17 : start
18 : when
19 : eyes
20 : green
21 : matter
22 : what
23 : with
24 : a
25 : book
26 : ever
27 : have
28 : read
29 : you
30 : bioinformatics
31 : can
32 : on
33 : try
34 : account
35 : are
36 : making
37 : this
38 : which
39 : withdrawal
40 : truth
41 : basketball
42 : dvd
43 : my
44 : watch
45 : we
46 : dad
47 : his
48 : nose
49 : saw
50 : sleeve
51 : wipe
52 : yesterday
53 : cancel
54 : for
55 : now
56 : right
57 : be
58 : day
59 : everything
60 : in
61 : m
62 : okay
63 : or
64 : sure
65 : two
66 : will
67 : actually
68 : been
69 : busy
70 : lately
71 : ve
72 : didn
73 : free
74 : go
75 : kind
76 : mad
77 : night
78 : of
79 : was
80 : wish
81 : doing
82 : how
83 : today
84 : about
85 : friend
86 : going
87 : movies
88 : to
89 : they
90 : do
91 : drink
92 : and
93 : goes
94 : price
95 : stamps
96 : up
97 : forecast
98 : says
99 : w

In [129]:
bow=[dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow):
    print(sent)
    print(embedding)

Streaming output truncated to the last 5000 lines.
i need pencils 
[(14, 1), (245, 1), (635, 1)]
oh  you mean like reuse things and recycle stuff 
[(29, 1), (92, 1), (111, 1), (136, 1), (172, 1), (187, 1), (1113, 1), (2061, 1), (2062, 1)]
they were about forty dollars 
[(84, 1), (89, 1), (309, 1), (685, 1), (2063, 1)]
why didn t she love you anymore 
[(4, 1), (29, 1), (72, 1), (152, 1), (215, 1), (321, 1), (323, 1)]
what does  mi  mean 
[(16, 1), (22, 1), (187, 1), (2064, 1)]
i ll put a glove in each pocket 
[(14, 1), (24, 1), (60, 1), (116, 1), (275, 1), (525, 1), (996, 1), (1797, 1)]
now he has the world record 
[(5, 1), (55, 1), (153, 1), (451, 1), (478, 1), (1774, 1)]
is it going to be perfect beach weather 
[(1, 1), (3, 1), (6, 1), (57, 1), (86, 1), (88, 1), (546, 1), (846, 1)]
not yet 
[(124, 1), (1106, 1)]
don t ever be in a hurry when you re driving 
[(4, 1), (18, 1), (24, 1), (26, 1), (29, 1), (57, 1), (60, 1), (177, 1), (258, 1), (1026, 1), (2065, 1)]
just fine  where are you

In [130]:
Question_orig='When Abraham Lincoln started his political career'
question=clean_sentence(Question_orig,stopwords=False)
question_embedding=dictionary.doc2bow(question.split())
print('\n\n',question,'\n',question_embedding)



 when abraham lincoln started his political career 
 [(18, 1), (47, 1), (1438, 1), (1539, 1)]


In [131]:
def nearest_question(question_embedding,sentence_embedding,ques,sentences):
    max_simalirty=-1;
    index_simalrit=-1;
    for index,answer_embedding in enumerate(sentence_embedding):
        simlarity=cosine_similarity(answer_embedding,question_embedding)[0][0];
        print(index,simlarity,sentences[index])
        #which question max simalryt and sort them
        if simlarity>max_simalirty:
            max_simalirty=simlarity;
            index_simalrity=index;
    print('\n')
    print('question',question)
    print('\n')
    print('retrived',ques.iloc[index_simalrity,0])
    print(ques.iloc[index_simalrity,1])
nearest_question(question_embedding,bow,data,sentences)

0 0.05547001962252291 the weather is great isn t it 
1 0.9899494936611666 that one  the one that s all black 
2 0.9647638212377322 i got it from macy s 
3 0.9647638212377322 when does it start 
4 0.9899494936611666 what s the matter with green eyes 
5 0.99990400819108 have you ever read a book
6 0.7452413135250993 what is bioinformatics
7 0.9647638212377322 can i try it on 
8 0.9996179641595869 which account are you making this withdrawal from 
9 0.9899494936611666 that s the truth 
10 0.7452413135250993 what is basketball
11 0.9997296931968458 we can watch my dvd 
12 0.9998750410016836 i saw dad wipe his nose on his sleeve yesterday 
13 0.9647638212377322 i can cancel it for you right now 
14 0.9998750410016836 i m sure everything will be okay in a day or two 
15 0.9998750410016836 i ve actually been busy lately 
16 0.9821022792346163 i wish i was free that night  i m kind of mad that i didn t go 
17 0.9997788902272606 how are you doing today 
18 0.9899494936611666 i m going to the mo

,I.D.,Question
0,QN_1,"i'll give you a speech like that, too."
1,QN_2,"i know, you're absolutely right."
2,QN_3,i liked it.
3,QN_4,the baby was eight pounds six ounces.
4,QN_5,I was sold a wireless service unavailable in m...


In [51]:
import os
os.chdir("/content/drive/MyDrive/nlp project")

In [66]:
file = open("TrainData.txt","r",encoding='utf-8')
train=file.read()

In [65]:
data

'Answer\tQuestion\nYes. It\'s absolutely beautiful today.\tThe weather is great isn\'t it?\nyes, i like that one, too.\tthat one. the one that\'s all black.\nit\'s really nice.\ti got it from macy\'s.\nat 8:00 p.m.\twhen does it start?\nnothing, except my favorite color is blue.\twhat\'s the matter with green eyes?\n I have read just about everything in Project Gutenberg.\t have you ever read a book\n a fancy name for applied computer science in biology.\t what is bioinformatics\nSure, the changing rooms are over there.\tCan I try it on?\nI want it taken from my checking account.\tWhich account are you making this withdrawal from?\nand you get to play with a lot of dogs.\tthat\'s the truth.\n A game with tall players.\t WHAT IS BASKETBALL\nand then we can go to my home.\twe can watch my dvd.\ni will talk to your father about that.\ti saw dad wipe his nose on his sleeve yesterday.\nThank you so much.\tI can cancel it for you right now.\nit\'s your girlfriend\'s fault. she should have wa

In [68]:
train

'Answer\tQuestion\nYes. It\'s absolutely beautiful today.\tThe weather is great isn\'t it?\nyes, i like that one, too.\tthat one. the one that\'s all black.\nit\'s really nice.\ti got it from macy\'s.\nat 8:00 p.m.\twhen does it start?\nnothing, except my favorite color is blue.\twhat\'s the matter with green eyes?\n I have read just about everything in Project Gutenberg.\t have you ever read a book\n a fancy name for applied computer science in biology.\t what is bioinformatics\nSure, the changing rooms are over there.\tCan I try it on?\nI want it taken from my checking account.\tWhich account are you making this withdrawal from?\nand you get to play with a lot of dogs.\tthat\'s the truth.\n A game with tall players.\t WHAT IS BASKETBALL\nand then we can go to my home.\twe can watch my dvd.\ni will talk to your father about that.\ti saw dad wipe his nose on his sleeve yesterday.\nThank you so much.\tI can cancel it for you right now.\nit\'s your girlfriend\'s fault. she should have wa

In [69]:
from bs4 import BeautifulSoup

In [70]:
example1 = BeautifulSoup(train)

In [73]:
len(example1)

1

In [74]:
import re
letters_only = re.sub("[^a-zA-Z]"," ",example1.get_text() )
print (letters_only)

Answer Question Yes  It s absolutely beautiful today  The weather is great isn t it  yes  i like that one  too  that one  the one that s all black  it s really nice  i got it from macy s  at      p m  when does it start  nothing  except my favorite color is blue  what s the matter with green eyes   I have read just about everything in Project Gutenberg   have you ever read a book  a fancy name for applied computer science in biology   what is bioinformatics Sure  the changing rooms are over there  Can I try it on  I want it taken from my checking account  Which account are you making this withdrawal from  and you get to play with a lot of dogs  that s the truth   A game with tall players   WHAT IS BASKETBALL and then we can go to my home  we can watch my dvd  i will talk to your father about that  i saw dad wipe his nose on his sleeve yesterday  Thank you so much  I can cancel it for you right now  it s your girlfriend s fault  she should have warned you  i m sure everything will be ok

In [75]:
lower_case = letters_only.lower() 

In [76]:
words = lower_case.split()

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [77]:
from nltk.corpus import stopwords
print (stopwords.words("english") )

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [78]:
words = [w for w in words if not w in stopwords.words("english")]
print (words)

['answer', 'question', 'yes', 'absolutely', 'beautiful', 'today', 'weather', 'great', 'yes', 'like', 'one', 'one', 'one', 'black', 'really', 'nice', 'got', 'macy', 'p', 'start', 'nothing', 'except', 'favorite', 'color', 'blue', 'matter', 'green', 'eyes', 'read', 'everything', 'project', 'gutenberg', 'ever', 'read', 'book', 'fancy', 'name', 'applied', 'computer', 'science', 'biology', 'bioinformatics', 'sure', 'changing', 'rooms', 'try', 'want', 'taken', 'checking', 'account', 'account', 'making', 'withdrawal', 'get', 'play', 'lot', 'dogs', 'truth', 'game', 'tall', 'players', 'basketball', 'go', 'home', 'watch', 'dvd', 'talk', 'father', 'saw', 'dad', 'wipe', 'nose', 'sleeve', 'yesterday', 'thank', 'much', 'cancel', 'right', 'girlfriend', 'fault', 'warned', 'sure', 'everything', 'okay', 'day', 'two', 'actually', 'busy', 'lately', 'great', 'game', 'wish', 'free', 'night', 'kind', 'mad', 'go', 'fine', 'today', 'sure', 'yet', 'going', 'movies', 'friend', 'bites', 'capable', 'drink', 'think'

In [79]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))  

In [82]:
clean_review = review_to_words( train )
print (clean_review)

answer question yes absolutely beautiful today weather great yes like one one one black really nice got macy p start nothing except favorite color blue matter green eyes read everything project gutenberg ever read book fancy name applied computer science biology bioinformatics sure changing rooms try want taken checking account account making withdrawal get play lot dogs truth game tall players basketball go home watch dvd talk father saw dad wipe nose sleeve yesterday thank much cancel right girlfriend fault warned sure everything okay day two actually busy lately great game wish free night kind mad go fine today sure yet going movies friend bites capable drink think stamps used cost penny price stamps goes think perfect weather beach forecast says warm weekend takes half hour long bus ride interesting watch movie coffee still tastes okay good idea mixed together kind emergency oh really maybe called yes gets colder sun goes get warmer walk never used buy swiss cheese breakfast maybe 

In [96]:
len(clean_review)

193721

In [101]:
content_list = train.splitlines()


In [107]:
content_list

['Answer\tQuestion',
 "Yes. It's absolutely beautiful today.\tThe weather is great isn't it?",
 "yes, i like that one, too.\tthat one. the one that's all black.",
 "it's really nice.\ti got it from macy's.",
 'at 8:00 p.m.\twhen does it start?',
 "nothing, except my favorite color is blue.\twhat's the matter with green eyes?",
 ' I have read just about everything in Project Gutenberg.\t have you ever read a book',
 ' a fancy name for applied computer science in biology.\t what is bioinformatics',
 'Sure, the changing rooms are over there.\tCan I try it on?',
 'I want it taken from my checking account.\tWhich account are you making this withdrawal from?',
 "and you get to play with a lot of dogs.\tthat's the truth.",
 ' A game with tall players.\t WHAT IS BASKETBALL',
 'and then we can go to my home.\twe can watch my dvd.',
 'i will talk to your father about that.\ti saw dad wipe his nose on his sleeve yesterday.',
 'Thank you so much.\tI can cancel it for you right now.',
 "it's your g

In [102]:
qstn=[]
anwr=[]
for lst in content_list[1:]:
  q=lst.split('\t')[-1]
  a=lst.split('\t')[0]
  qstn.append(q)
  anwr.append(a)

In [103]:
qstn[:2]

["The weather is great isn't it?", "that one. the one that's all black."]

In [104]:
anwr[:2]

["Yes. It's absolutely beautiful today.", 'yes, i like that one, too.']

In [109]:
import numpy as np
import pandas as pd

In [110]:
df1=pd.DataFrame([qstn,anwr])

In [112]:
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,5455,5456,5457,5458,5459,5460,5461,5462,5463,5464,5465,5466,5467,5468,5469,5470,5471,5472,5473,5474,5475,5476,5477,5478,5479,5480,5481,5482,5483,5484,5485,5486,5487,5488,5489,5490,5491,5492,5493,5494
0,The weather is great isn't it?,that one. the one that's all black.,i got it from macy's.,when does it start?,what's the matter with green eyes?,have you ever read a book,what is bioinformatics,Can I try it on?,Which account are you making this withdrawal f...,that's the truth.,WHAT IS BASKETBALL,we can watch my dvd.,i saw dad wipe his nose on his sleeve yesterday.,I can cancel it for you right now.,i'm sure everything will be okay in a day or two.,i've actually been busy lately.,i wish i was free that night. i'm kind of mad ...,How are you doing today?,i'm going to the movies with a friend. how abo...,what are they?,do you drink,the price of stamps goes up and up.,the forecast says that it will be warm on the ...,How long is this bus ride?,How was the movie?,i mixed them together.,"oh, really? maybe you should have called 911.",we'll get warmer as we walk.,i never used to buy swiss cheese.,maybe. i'm not sure.,i lost my new pen.,you have no idea how happy i am for you.,"i have to do some things, and besides, it's no...",how did they survive in the old days?,what is it this time?,then you should watch the rerun.,but a house is so expensive.,then why does everyone lock their doors?,he thinks i have too much stress.,"no, i'd rather not find out that it's dirty, b...",...,Top of the morning to you!,"yes, it does. it's only $20.",how good is your math?,i like that shirt.,i have a little poodle.,guns,do you need a pen?,i hope i win the lotto.,i don't understand why you haven't come to vis...,I never realized.,Do you feel pain,"yes, old people don't smell like fruit.",i was on a plane.,Yeah!Just focus on the schools that do want you.,i went to the market yesterday.,What is your age?,it's based on a true story about a divorced ma...,of course i will.,do androids dream of electric sheep,I want sheets without stains on them.,"well, hold on a little longer.",winter is great. i wish it didn't get so cold ...,it only took about two hours.,what do you need for school?,what is the illuminatti,what does it mean?,Don't pull my chain.,and there are only six units in the whole buil...,what is your major?,what is the illuminatti,which would you prefer?,Tell me a joke,How much would you like to deposit?,the book is open.,They're very comfortable.,at night i went out to eat. the food was delic...,TELL ME ABOUT BASEBALL,"you'll remember to do it, but you won't have t...",you are not smart,what do you mean?
1,Yes. It's absolutely beautiful today.,"yes, i like that one, too.",it's really nice.,at 8:00 p.m.,"nothing, except my favorite color is blue.",I have read just about everything in Project ...,a fancy name for applied computer science in ...,"Sure, the changing rooms are over there.",I want it taken from my checking account.,and you get to play with a lot of dogs.,A game with tall players.,and then we can go to my home.,i will talk to your father about that.,Thank you so much.,it's your girlfriend's fault. she should have ...,what have you been doing?,it was a great game.,I am fine,i'm not sure yet.,they are bites.,I am not capable of doing so.,i think stamps used to cost a penny.,so do you think it'll be perfect weather for t...,It only takes half an hour.,It was very interesting. You should watch it.,"if the coffee still tastes okay, that's a good...",it wasn't that kind of emergency.,"yes, but it gets colder as the sun goes down.",why not?,what did you have for breakfast?,where did you lose it?,for real?,i'm not being nosey. i'm just asking.,they had fireplaces.,an andy warhol drawing.,why? i'm watching the original.,that's why we need to save money.,i guess they haven't read this article.,stress causes your stomachaches?,"me, too. 